## Keras Implementation of a DQN

Trained on the game catcher. Dependencies:
* Keras
* TensorFlow
* Pygame Learning Environment: https://github.com/ntasfi/PyGame-Learning-Environment
* numpy

Code used from:
* Using Keras to solve FlappyBird: https://github.com/yanpanlau/Keras-FlappyBird
* Denny Britz's DQN implementation: https://github.com/dennybritz/reinforcement-learning/tree/master/DQN
* Udacity's RL Implementation: https://github.com/udacity/deep-learning/tree/master/reinforcement 

In [1]:
import numpy as np
from ple import PLE
from ple.games.catcher import Catcher

import numpy as np
from collections import deque
import random

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras import initializers
import keras.backend as K


LEARNING_RATE = 1e-4
ACTIONS = 2 #right, left
action_list = [97,100]
GAMMA = 0.99 # decay rate of past observations
EXPLORE = 4000000. #3000000. # frames over which to anneal epsilon, 
FINAL_EPSILON = 0.01 # final value of epsilon
INITIAL_EPSILON = 0.95 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 #32 # size of minibatch
#FRAME_PER_ACTION = 1 #set in PLE environement
LEARNING_RATE = 1e-4

img_rows , img_cols = 64, 64 #default size of catcher
#Convert image into Black and white
img_channels = 4 #stacking 4 frames

#DQN (https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf 2015, Minh) has different architecture
#using slight modifications from denny britz and flappy bird code
#modifying it with action length for this game
def build_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(8,8),strides=(4, 4), padding='same',
                     activation='relu', input_shape=(img_rows,img_cols,img_channels), 
                     kernel_initializer=initializers.glorot_normal(seed=31))) 
    model.add(Conv2D(64,kernel_size=(4,4),strides=(2,2),padding='same',activation='relu', 
                     kernel_initializer=initializers.glorot_normal(seed=31)))
    model.add(Conv2D(64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu', 
                     kernel_initializer=initializers.glorot_normal(seed=31)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu', 
                     kernel_initializer=initializers.glorot_normal(seed=31)))
    model.add(Dense(ACTIONS, 
                     kernel_initializer=initializers.glorot_normal(seed=31)))
    
    adam = Adam(lr=LEARNING_RATE,clipvalue=1.0)
    model.compile(loss='mse',optimizer=adam)
    return model


couldn't import doomish
Couldn't import doom


Using TensorFlow backend.


In [2]:
def train_network(model,target_model):
    
    REPORT_EVERY = 2000 #report results this often
    UPDATE_EVERY = 10000 #update target_model after this many training steps
    
    # store the previous observations in replay memory
    exp_replay = deque()
    
    #stop training after hitting max loops
    train_loops = 0
    train_loops_max = 10
    
    loss = 0
    Q_sa = 0
    action_index = 0 #action index stored in experience replay
    action_command = action_list[action_index] #actual command given to the environment
    r_t = 0 #reward
    epsilon = INITIAL_EPSILON
    s_t = None #state
    s_t1 = None #resulting state
    skip_append = True

    ti_tuple = tuple([i for i in range(BATCH)]) #used for indexing a np array down below, probably a better way to do this
    
    EPISODE_DONE = 0 #0 if done, 1 if continuing makes math easier when making targets
    EPISODE_NOT_DONE = 1 #makes math easier
    EPISODE_START = -1
    episode_length = 0.
    episode_reward = 0.
    stats_episode_avg_reward = 0.
    stats_episode_avg_length = 0.
    stats_episode = 0
    episode = 1
    turn = 0
    
    game = Catcher() # create our game

    fps = 30  # fps we want to run at
    frame_skip = 2
    num_steps = 2
    force_fps = True #True # false for slower speed
    display_screen = False

    # make a PLE instance.
    ple_env = PLE(game, fps=fps, frame_skip=frame_skip, num_steps=num_steps,
            force_fps=force_fps, display_screen=display_screen)

    # init agent and game.
    ple_env.init()
    done_check = ple_env.lives() #resetting episode on loss of life, not on game over
    ple_env.act(action_list[np.random.randint(0,2)]) #need to take an action to initialize, screen starts out as black
    
    #start of an episode is a stack of four of the same observation
    #subsequent time steps removes the first frame adds current frame to the end
    x_t = ple_env.getScreenGrayscale()/255. 
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)

    while train_loops < EXPLORE:
        train_loops += 1
        
        #get action
        #action_command used to actually input the command
        #a_t is used in the experience replay/training
        if random.random() <= epsilon:
            action_index = np.random.randint(0,ACTIONS)
            action_command = action_list[action_index]
        else:
            q = model.predict(s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])) 
            max_Q = np.argmax(q)
            action_index = max_Q
            action_command = action_list[action_index]
            
        #reduce exploration rate epsilon
        if epsilon > FINAL_EPSILON:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE
        
        #do action
        r_t = ple_env.act(action_command)
        episode_reward += r_t
        episode_length += 1
        
        #get next state
        x_t1 = ple_env.getScreenGrayscale()/255.
        x_t1 = x_t1.reshape(x_t1.shape[0], x_t1.shape[1], 1) #1x64x64x1
        s_t1 = np.append(x_t1, s_t[ :, :, :3], axis=2)

        if ple_env.lives() != done_check:
            done = EPISODE_DONE
        else:
            done = EPISODE_NOT_DONE
        
        #adding to experience replay
        if len(exp_replay) == REPLAY_MEMORY:
            exp_replay.popleft()
        exp_replay.append((s_t, action_index, r_t, s_t1, done))
        
        #check if life is lost, if so episode is done and we reset
        if done == EPISODE_DONE:
            episode += 1
            stats_episode += 1
            turn = 0
            stats_episode_avg_reward = stats_episode_avg_reward + (episode_reward - stats_episode_avg_reward)/stats_episode
            stats_episode_avg_length = stats_episode_avg_length + (episode_length - stats_episode_avg_length)/stats_episode
            episode_reward = 0.
            episode_length = 0.

            #reset the environment, get the current state
            ple_env.reset_game()
            ple_env.act(action_list[np.random.randint(0,2)]) #take action to initialize, screen starts out as black
            x_t = ple_env.getScreenGrayscale()/255.
            s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
        else:
            s_t = s_t1 #resulting state becomes the current state
        
        #train the network using the experience replay
        #modified heavily from FlappyBird implementation
            #better way from dennybritz design
                #used a fixed Q target network
                #only predicts on the next values, flappy bird predicts on current values which I don't think the paper does
                #removes a for loop
                
        if len(exp_replay) > BATCH:
            #print("training")
            #minibatch = exp_replay.sample(BATCHD)
            minibatch = random.sample(exp_replay,BATCH)
            
            #modified heavily from flapply bird way which I believe is incorrect
            #better way from dennybritz design
                #uses a fixed Q target network
                #only predicts on the next states values
                    #flappy bird predicts on current values which I don't think the DQN paper does
                #removes a for loop

            states_batch, action_batch, reward_batch, next_states_batch, done_batch = map(np.array, zip(*minibatch))        
            
            q_values_next = target_model.predict(next_states_batch,batch_size=BATCH)
            targets = np.zeros((BATCH,ACTIONS)) #BATCHxACTIONS
            targets[ti_tuple,action_batch] = reward_batch + done_batch * GAMMA * np.amax(q_values_next,axis=1)
            loss += model.train_on_batch(states_batch, targets)
        
        #save model weights and update target model
        if train_loops % UPDATE_EVERY == 0:
            print("Saving model weights")
            zString = "catcher_training_weights/model_{}.h5".format(train_loops)
            model.save_weights(zString, overwrite=True)
            #updating fixed Q network weights
            target_model.load_weights(zString)

        #print info
        if train_loops % REPORT_EVERY == 0:
            print("Loop {} / Episode {} / Epsilon {:6.4f} / Avg. Reward {:6.3f} / Avg. Length {:6.3f} / Loss {:6.4f} "
                              .format(train_loops,episode,epsilon,stats_episode_avg_reward,stats_episode_avg_length,loss) )
            stats_episode_avg_reward = 0.
            stats_episode_avg_length = 0.
            stats_episode = 0.
            loss = 0.
        

In [3]:
model = build_model()
target_model = build_model()

train_network(model,target_model)

Loop 2000 / Episode 156 / Epsilon 0.9495 / Avg. Reward -0.548 / Avg. Length 12.884 / Loss 21.7476 
Loop 4000 / Episode 304 / Epsilon 0.9491 / Avg. Reward -0.480 / Avg. Length 13.486 / Loss 19.8332 
Loop 6000 / Episode 459 / Epsilon 0.9486 / Avg. Reward -0.542 / Avg. Length 12.897 / Loss 21.2444 
Loop 8000 / Episode 625 / Epsilon 0.9481 / Avg. Reward -0.651 / Avg. Length 12.054 / Loss 21.3462 
Saving model weights
Loop 10000 / Episode 790 / Epsilon 0.9476 / Avg. Reward -0.624 / Avg. Length 12.121 / Loss 20.8022 
Loop 12000 / Episode 940 / Epsilon 0.9472 / Avg. Reward -0.480 / Avg. Length 13.340 / Loss 35.8715 
Loop 14000 / Episode 1091 / Epsilon 0.9467 / Avg. Reward -0.497 / Avg. Length 13.245 / Loss 29.2353 
Loop 16000 / Episode 1230 / Epsilon 0.9462 / Avg. Reward -0.367 / Avg. Length 14.410 / Loss 29.5682 
Loop 18000 / Episode 1387 / Epsilon 0.9458 / Avg. Reward -0.567 / Avg. Length 12.732 / Loss 27.6158 
Saving model weights
Loop 20000 / Episode 1553 / Epsilon 0.9453 / Avg. Reward -0

Loop 158000 / Episode 12224 / Epsilon 0.9129 / Avg. Reward -0.452 / Avg. Length 13.733 / Loss 47.2238 
Saving model weights
Loop 160000 / Episode 12392 / Epsilon 0.9124 / Avg. Reward -0.649 / Avg. Length 11.988 / Loss 45.9708 
Loop 162000 / Episode 12548 / Epsilon 0.9119 / Avg. Reward -0.551 / Avg. Length 12.846 / Loss 51.9891 
Loop 164000 / Episode 12699 / Epsilon 0.9115 / Avg. Reward -0.503 / Avg. Length 13.252 / Loss 50.9671 
Loop 166000 / Episode 12845 / Epsilon 0.9110 / Avg. Reward -0.445 / Avg. Length 13.678 / Loss 47.3807 
Loop 168000 / Episode 12989 / Epsilon 0.9105 / Avg. Reward -0.410 / Avg. Length 13.896 / Loss 46.8538 
Saving model weights
Loop 170000 / Episode 13139 / Epsilon 0.9100 / Avg. Reward -0.513 / Avg. Length 13.327 / Loss 46.5756 
Loop 172000 / Episode 13296 / Epsilon 0.9096 / Avg. Reward -0.567 / Avg. Length 12.752 / Loss 50.5967 
Loop 174000 / Episode 13454 / Epsilon 0.9091 / Avg. Reward -0.563 / Avg. Length 12.620 / Loss 48.1845 
Loop 176000 / Episode 13604 / E

Loop 312000 / Episode 23931 / Epsilon 0.8767 / Avg. Reward -0.545 / Avg. Length 12.763 / Loss 56.0301 
Loop 314000 / Episode 24080 / Epsilon 0.8762 / Avg. Reward -0.477 / Avg. Length 13.530 / Loss 54.0700 
Loop 316000 / Episode 24234 / Epsilon 0.8757 / Avg. Reward -0.526 / Avg. Length 12.961 / Loss 52.0396 
Loop 318000 / Episode 24379 / Epsilon 0.8753 / Avg. Reward -0.441 / Avg. Length 13.779 / Loss 51.5743 
Saving model weights
Loop 320000 / Episode 24523 / Epsilon 0.8748 / Avg. Reward -0.431 / Avg. Length 13.826 / Loss 50.6085 
Loop 322000 / Episode 24680 / Epsilon 0.8743 / Avg. Reward -0.548 / Avg. Length 12.726 / Loss 54.9213 
Loop 324000 / Episode 24828 / Epsilon 0.8739 / Avg. Reward -0.459 / Avg. Length 13.595 / Loss 52.5702 
Loop 326000 / Episode 24980 / Epsilon 0.8734 / Avg. Reward -0.539 / Avg. Length 13.020 / Loss 51.4520 
Loop 328000 / Episode 25140 / Epsilon 0.8729 / Avg. Reward -0.594 / Avg. Length 12.500 / Loss 49.6369 
Saving model weights
Loop 330000 / Episode 25292 / E

Loop 466000 / Episode 35343 / Epsilon 0.8405 / Avg. Reward -0.455 / Avg. Length 13.717 / Loss 51.1757 
Loop 468000 / Episode 35487 / Epsilon 0.8400 / Avg. Reward -0.417 / Avg. Length 13.951 / Loss 49.5005 
Saving model weights
Loop 470000 / Episode 35631 / Epsilon 0.8395 / Avg. Reward -0.431 / Avg. Length 13.903 / Loss 49.2212 
Loop 472000 / Episode 35780 / Epsilon 0.8391 / Avg. Reward -0.490 / Avg. Length 13.416 / Loss 54.9944 
Loop 474000 / Episode 35922 / Epsilon 0.8386 / Avg. Reward -0.401 / Avg. Length 14.077 / Loss 52.1767 
Loop 476000 / Episode 36067 / Epsilon 0.8381 / Avg. Reward -0.421 / Avg. Length 13.828 / Loss 51.0660 
Loop 478000 / Episode 36224 / Epsilon 0.8377 / Avg. Reward -0.567 / Avg. Length 12.720 / Loss 51.3465 
Saving model weights
Loop 480000 / Episode 36372 / Epsilon 0.8372 / Avg. Reward -0.480 / Avg. Length 13.385 / Loss 49.3943 
Loop 482000 / Episode 36526 / Epsilon 0.8367 / Avg. Reward -0.519 / Avg. Length 13.104 / Loss 52.3315 
Loop 484000 / Episode 36684 / E

Saving model weights
Loop 620000 / Episode 46324 / Epsilon 0.8043 / Avg. Reward -0.386 / Avg. Length 14.271 / Loss 52.7638 
Loop 622000 / Episode 46461 / Epsilon 0.8038 / Avg. Reward -0.358 / Avg. Length 14.555 / Loss 55.4667 
Loop 624000 / Episode 46599 / Epsilon 0.8034 / Avg. Reward -0.348 / Avg. Length 14.594 / Loss 53.5830 
Loop 626000 / Episode 46730 / Epsilon 0.8029 / Avg. Reward -0.290 / Avg. Length 15.160 / Loss 52.4945 
Loop 628000 / Episode 46881 / Epsilon 0.8024 / Avg. Reward -0.497 / Avg. Length 13.278 / Loss 51.7810 
Saving model weights
Loop 630000 / Episode 47029 / Epsilon 0.8019 / Avg. Reward -0.473 / Avg. Length 13.399 / Loss 49.7541 
Loop 632000 / Episode 47149 / Epsilon 0.8015 / Avg. Reward -0.092 / Avg. Length 16.817 / Loss 55.5396 
Loop 634000 / Episode 47296 / Epsilon 0.8010 / Avg. Reward -0.449 / Avg. Length 13.646 / Loss 52.2627 
Loop 636000 / Episode 47437 / Epsilon 0.8005 / Avg. Reward -0.411 / Avg. Length 14.078 / Loss 50.9482 
Loop 638000 / Episode 47575 / E

...

Loop 3348000 / Episode 168595 / Epsilon 0.1632 / Avg. Reward  4.970 / Avg. Length 59.576 / Loss 159.3275 
Saving model weights
Loop 3350000 / Episode 168633 / Epsilon 0.1627 / Avg. Reward  4.184 / Avg. Length 53.263 / Loss 162.1569 
Loop 3352000 / Episode 168663 / Epsilon 0.1623 / Avg. Reward  5.900 / Avg. Length 67.100 / Loss 167.1681 
Loop 3354000 / Episode 168698 / Epsilon 0.1618 / Avg. Reward  4.743 / Avg. Length 57.486 / Loss 163.6308 
Loop 3356000 / Episode 168723 / Epsilon 0.1613 / Avg. Reward  7.080 / Avg. Length 77.640 / Loss 164.4156 
Loop 3358000 / Episode 168763 / Epsilon 0.1609 / Avg. Reward  4.050 / Avg. Length 51.600 / Loss 161.7446 
Saving model weights
Loop 3360000 / Episode 168805 / Epsilon 0.1604 / Avg. Reward  3.548 / Avg. Length 47.357 / Loss 163.3915 
Loop 3362000 / Episode 168837 / Epsilon 0.1599 / Avg. Reward  4.906 / Avg. Length 58.656 / Loss 163.4999 
Loop 3364000 / Episode 168872 / Epsilon 0.1595 / Avg. Reward  5.086 / Avg. Length 61.143 / Loss 160.6196 
Loop

Loop 3498000 / Episode 170760 / Epsilon 0.1280 / Avg. Reward  5.774 / Avg. Length 66.806 / Loss 180.7564 
Saving model weights
Loop 3500000 / Episode 170783 / Epsilon 0.1275 / Avg. Reward  7.217 / Avg. Length 78.870 / Loss 181.5934 
Loop 3502000 / Episode 170811 / Epsilon 0.1270 / Avg. Reward  7.000 / Avg. Length 77.321 / Loss 186.2990 
Loop 3504000 / Episode 170839 / Epsilon 0.1266 / Avg. Reward  5.929 / Avg. Length 67.429 / Loss 180.2319 
Loop 3506000 / Episode 170868 / Epsilon 0.1261 / Avg. Reward  6.276 / Avg. Length 71.103 / Loss 184.2708 
Loop 3508000 / Episode 170900 / Epsilon 0.1256 / Avg. Reward  5.156 / Avg. Length 61.094 / Loss 177.4814 
Saving model weights
Loop 3510000 / Episode 170929 / Epsilon 0.1251 / Avg. Reward  6.448 / Avg. Length 71.793 / Loss 184.9417 
Loop 3512000 / Episode 170949 / Epsilon 0.1247 / Avg. Reward  9.350 / Avg. Length 96.750 / Loss 189.8010 
Loop 3514000 / Episode 170971 / Epsilon 0.1242 / Avg. Reward  8.773 / Avg. Length 92.182 / Loss 188.3912 
Loop

Loop 3648000 / Episode 172421 / Epsilon 0.0927 / Avg. Reward 11.765 / Avg. Length 117.941 / Loss 192.8348 
Saving model weights
Loop 3650000 / Episode 172444 / Epsilon 0.0922 / Avg. Reward  7.391 / Avg. Length 80.217 / Loss 194.4802 
Loop 3652000 / Episode 172461 / Epsilon 0.0918 / Avg. Reward 12.824 / Avg. Length 125.824 / Loss 200.5198 
Loop 3654000 / Episode 172480 / Epsilon 0.0913 / Avg. Reward  9.737 / Avg. Length 99.368 / Loss 192.1170 
Loop 3656000 / Episode 172501 / Epsilon 0.0908 / Avg. Reward  9.952 / Avg. Length 102.524 / Loss 194.9035 
Loop 3658000 / Episode 172518 / Epsilon 0.0904 / Avg. Reward 11.647 / Avg. Length 116.882 / Loss 193.5189 
Saving model weights
Loop 3660000 / Episode 172539 / Epsilon 0.0899 / Avg. Reward  8.810 / Avg. Length 92.905 / Loss 191.7188 
Loop 3662000 / Episode 172554 / Epsilon 0.0894 / Avg. Reward 14.067 / Avg. Length 137.533 / Loss 202.9002 
Loop 3664000 / Episode 172576 / Epsilon 0.0890 / Avg. Reward  8.364 / Avg. Length 88.091 / Loss 201.2295 

Loop 3796000 / Episode 173557 / Epsilon 0.0579 / Avg. Reward 36.000 / Avg. Length 323.000 / Loss 219.1211 
Loop 3798000 / Episode 173571 / Epsilon 0.0575 / Avg. Reward 16.500 / Avg. Length 157.857 / Loss 222.4497 
Saving model weights
Loop 3800000 / Episode 173587 / Epsilon 0.0570 / Avg. Reward 12.625 / Avg. Length 124.125 / Loss 219.3792 
Loop 3802000 / Episode 173600 / Epsilon 0.0565 / Avg. Reward 14.692 / Avg. Length 142.769 / Loss 234.6546 
Loop 3804000 / Episode 173614 / Epsilon 0.0561 / Avg. Reward 15.071 / Avg. Length 146.214 / Loss 224.5973 
Loop 3806000 / Episode 173630 / Epsilon 0.0556 / Avg. Reward 13.375 / Avg. Length 131.562 / Loss 222.3192 
Loop 3808000 / Episode 173639 / Epsilon 0.0551 / Avg. Reward 22.000 / Avg. Length 203.444 / Loss 219.3355 
Saving model weights
Loop 3810000 / Episode 173650 / Epsilon 0.0546 / Avg. Reward 21.091 / Avg. Length 196.000 / Loss 215.5241 
Loop 3812000 / Episode 173660 / Epsilon 0.0542 / Avg. Reward 20.400 / Avg. Length 190.900 / Loss 221.2

Loop 3944000 / Episode 174219 / Epsilon 0.0232 / Avg. Reward 65.750 / Avg. Length 572.500 / Loss 177.3754 
Loop 3946000 / Episode 174223 / Epsilon 0.0227 / Avg. Reward 58.500 / Avg. Length 513.000 / Loss 175.7493 
Loop 3948000 / Episode 174229 / Epsilon 0.0222 / Avg. Reward 38.167 / Avg. Length 339.667 / Loss 177.9578 
Saving model weights
Loop 3950000 / Episode 174234 / Epsilon 0.0217 / Avg. Reward 42.200 / Avg. Length 374.800 / Loss 168.8184 
Loop 3952000 / Episode 174240 / Epsilon 0.0213 / Avg. Reward 36.000 / Avg. Length 324.833 / Loss 171.9087 
Loop 3954000 / Episode 174245 / Epsilon 0.0208 / Avg. Reward 31.400 / Avg. Length 287.400 / Loss 175.8589 
Loop 3956000 / Episode 174252 / Epsilon 0.0203 / Avg. Reward 32.429 / Avg. Length 293.143 / Loss 178.6128 
Loop 3958000 / Episode 174257 / Epsilon 0.0199 / Avg. Reward 58.800 / Avg. Length 513.200 / Loss 179.7225 
Saving model weights
Loop 3960000 / Episode 174262 / Epsilon 0.0194 / Avg. Reward 43.200 / Avg. Length 385.600 / Loss 163.4

In [62]:
#running the model

inference_model = build_model()
zString = "catcher_training_weights/model_3000000.h5"
#zString = "catcher_training_weights/model_2950000.h5"
inference_model.load_weights(zString)

game = Catcher() # create our game

fps = 30  # fps we want to run at
frame_skip = 2
num_steps = 2
force_fps = False
display_screen = True

# make a PLE instance.
ple_env = PLE(game, fps=fps, frame_skip=frame_skip, num_steps=num_steps,
        force_fps=force_fps, display_screen=display_screen)

# init agent and game.
ple_env.init()
ple_env.act(action_list[np.random.randint(0,2)])

done_check = ple_env.lives() 

x_t = ple_env.getScreenGrayscale()/255. 
s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)

total_reward = 0
total_steps = 0

for f in range(1000000):
    q = inference_model.predict(s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])) 
    max_Q = np.argmax(q)
    action_index = max_Q
    action_command = action_list[action_index]
            
    total_reward += ple_env.act(action_command)
    total_steps += 1
    
    #get next state
    x_t1 = ple_env.getScreenGrayscale()/255.
    x_t1 = x_t1.reshape(x_t1.shape[0], x_t1.shape[1], 1) #1x64x64x1
    s_t1 = np.append(x_t1, s_t[ :, :, :3], axis=2) #add new observation to end, drop first observation
    
    # if the game is over
    if ple_env.lives() != done_check:
        print("Life lost: reward {}, steps {}".format(total_reward,total_steps))
        ple_env.reset_game()
        ple_env.act(action_list[np.random.randint(0,2)])
        x_t = ple_env.getScreenGrayscale()/255. 
        s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
        total_reward = 0
        total_steps = 0
    else:
        s_t = s_t1
    

Life lost: reward 18.0, steps 169
Life lost: reward 84.0, steps 733
Life lost: reward 20.0, steps 185
Life lost: reward 60.0, steps 526
Life lost: reward 69.0, steps 600
Life lost: reward 23.0, steps 214
Life lost: reward 44.0, steps 392
Life lost: reward 32.0, steps 282
Life lost: reward 11.0, steps 109
Life lost: reward 98.0, steps 842
Life lost: reward 119.0, steps 1033
Life lost: reward 93.0, steps 809
Life lost: reward 94.0, steps 826
Life lost: reward 97.0, steps 849
Life lost: reward 71.0, steps 622
Life lost: reward 194.0, steps 1657
Life lost: reward 90.0, steps 792
Life lost: reward 65.0, steps 576
Life lost: reward 89.0, steps 768
Life lost: reward 12.0, steps 119
Life lost: reward 13.0, steps 124
Life lost: reward 320.0, steps 2726
Life lost: reward 58.0, steps 504
Life lost: reward 59.0, steps 511
Life lost: reward 174.0, steps 1497
Life lost: reward 71.0, steps 623
Life lost: reward 43.0, steps 378
Life lost: reward 106.0, steps 917
Life lost: reward 95.0, steps 827
Life 

KeyboardInterrupt: 

### Notes:
* Still kind of not great considering catcher is such a simple game
* Could use some hyperparameter adjustments
* Does weird thing where it prefers all the way to the right then shifts left even when not needed. This could be because DQN is deterministic and a stochastic solution (like a Policy Gradients method) would lead to a better solution